# Import packages

First, we import the necessary packages for exploring the data.

In [ ]:
import xarray as xr
import matplotlib.pyplot as plt
import cartopy.crs as ccrs
import numpy as np
import cartopy.feature as cfeature

# Load in the dataset(s)

This functions preprocesses thet loaded datasets:
- It selects the period of interest.
- It changes the units of precipitation and temperature.

In [ ]:
def preprocess_dataset(ds, tstart, tstop):
    ds = ds.sel(time=slice(tstart, tstop))
    tas_vars = ["tas", "tasmax", "tasmin"]
    for tas_var in tas_vars:
        if tas_var in ds:
            ds[tas_var].values = ds[tas_var].values - 273.15
            ds[tas_var].attrs["units"] = "°C"

    return ds

In [ ]:
run_name = "baseline"
run_info_list = [
    ("2019070100", 720),
    ("2019071600", 360),
    ("2019071612", 348),
    ("2019071700", 336),
    ("2019071712", 324),
    ("2019071800", 312),
    ("2019071812", 300),
    ("2019071900", 288),
    ("2019071912", 276),
    ("2019072000", 264),
]
tstart = "2019-07-22T01"
tstop = "2019-07-28T00"

In [ ]:
ds_list = list()

for run_info in run_info_list:
    rstart, nhours = run_info
    filepath = f"../data/{run_name}_{rstart}_{nhours}*.nc"
    ds = xr.open_mfdataset(filepath, engine="netcdf4", chunks="auto").assign_coords(
        rstart=rstart
    )
    ds = preprocess_dataset(ds, tstart, tstop)
    ds_list.append(ds)

ds = xr.concat(ds_list, dim="rstart")
ds.load()

# Maxmimum temperature during period

During the period, what is the maximum temperature in each point?

In [ ]:
ncols = len(run_info_list)
fig, axs = plt.subplots(
    ncols=ncols,
    figsize=(ncols * 3.2, 4.8),
    layout="constrained",
    subplot_kw={"projection": ccrs.PlateCarree()},
)

for idx in range(len(run_info_list)):
    rstart, _ = run_info_list[idx]
    ax = axs[idx]
    p = (
        ds.sel(rstart=rstart)
        .tasmax.max(dim=["time"])
        .plot(
            ax=ax,
            cmap="coolwarm",
            levels=np.arange(20, 41, 2),
            extend="both",
            add_colorbar=False,
        )
    )
    ax.set_title(
        f"rstart = {rstart}\nmaximum temperature = {ds.sel(rstart=rstart).tasmax.max().values:.2f} °C"
    )
    ax.coastlines(alpha=0.7)
    ax.add_feature(cfeature.BORDERS, alpha=0.5, linestyle="dotted")
    ax.set_aspect(1.4)

fig.suptitle(
    f"Maximum temperature in each point between {tstart} and {tstop}",
    fontsize="xx-large",
)
fig.colorbar(p, label="Temperature [°C]")
plt.show()

There are small differences between tasmax.max() and tas.max(). Because of regridding?

# Average maximum temperature

In [ ]:
ncols = len(run_info_list)
fig, axs = plt.subplots(
    ncols=ncols,
    figsize=(ncols * 3.2, 4.8),
    layout="constrained",
    subplot_kw={"projection": ccrs.PlateCarree()},
)

for idx in range(len(run_info_list)):
    rstart, _ = run_info_list[idx]
    ax = axs[idx]
    p = (
        ds.sel(rstart=rstart)
        .tasmax.mean(dim=["time"])
        .plot(
            ax=ax,
            cmap="coolwarm",
            levels=np.arange(20, 31, 1),
            extend="both",
            add_colorbar=False,
        )
    )
    ax.set_title(
        f"rstart = {rstart}\nmean max temperature = {ds.sel(rstart=rstart).tasmax.mean().values:.2f} °C"
    )
    ax.coastlines(alpha=0.7)
    ax.add_feature(cfeature.BORDERS, alpha=0.5, linestyle="dotted")
    ax.set_aspect(1.4)

fig.suptitle(
    f"Mean max temperature in each point between {tstart} and {tstop}",
    fontsize="xx-large",
)
fig.colorbar(p, label="Temperature [°C]")
plt.show()

# Number of locations with a heatwave

In [ ]:
ncols = len(run_info_list)
fig, axs = plt.subplots(ncols=ncols, figsize=(ncols * 3.2, 4.8), layout="constrained")

for idx in range(len(run_info_list)):
    rstart, _ = run_info_list[idx]
    ax = axs[idx]

    tasmax = ds.tasmax.sel(rstart=rstart).dropna(dim="time")
    heatwave_mask_1 = (
        tasmax.rolling(time=5).construct("window_dim").min("window_dim", skipna=False)
        >= 25
    )
    heatwave_mask_2 = (
        tasmax.rolling(time=3).construct("window_dim").min("window_dim", skipna=False)
        >= 30
    )
    heatwave_mask = (heatwave_mask_1 & heatwave_mask_2).any("time")
    tasmax_masked = tasmax.where(heatwave_mask)

    tasmax_heatwave_locs = (
        tasmax_masked.stack(loc=["lat", "lon"]).dropna("loc").load().reset_index("loc")
    )
    tasmax_heatwave_locs.plot(ax=ax, x="time", hue="loc")
    ax.set_title(f"rstart = {rstart}; n_loc = {len(tasmax_heatwave_locs['loc'])}")
    ax.legend("").remove()
    ax.set_ylim(19, 41)

plt.show()